# Network Analysis of Functional Connectivity

In this tutorial, we'll apply simply graph theoretical techniques to characterize our functional connectivity data we generated in tutorial 10. We'll discuss how our connectivity matrix maps onto a graph, and what types of measures we can pull out of it in order to understand the underlying connectivity structure. 

Without a doubt, the foundational paper for the analyses we'll perfom here is [Rubinov and Sporns (2010)](https://www.sciencedirect.com/science/article/abs/pii/S105381190901074X?via%3Dihub). They describe a variety of ways in which we can analyze brain networks beyond applying simple statistics to connectivity matrices (e.g., mass univariate testing). Definitely required reading if you are interested in functional connectivity. Another excellent resource is [Fundamentals of Brain Network Analysis](https://www.elsevier.com/books/fundamentals-of-brain-network-analysis/fornito/978-0-12-407908-3) by Bullmore et al.

To get started, we need to install [Networkx](https://networkx.github.io/documentation/stable/index.html), which let's us generate graphs. In combination with Brain Connectivity Toolbox (BCT), we have a core toolbox for network neuroscience at our disposal. Installing it below:

In [ ]:
pip install networkx

Like last time, make sure you restart the notebook kernel (`Kernel` > `Restart Kernel...`). Once that is done, you can run the next cell to import all of the dependencies for today. 

In [ ]:
import numpy as np
import pandas as pd
from scipy import spatial, stats, cluster
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
from nilearn import plotting, input_data
from nilearn.datasets import fetch_atlas_schaefer_2018
import networkx as nx
import bct

We'll load in our atlas we used in tutorial 10 (the Schaefer 100 atlas). We'll be using this later. 

In [ ]:
atlas = fetch_atlas_schaefer_2018(n_rois=100, resolution_mm=2)
labels = [x.decode() for x in atlas['labels']]

## 1. Thresholding

First, we'll load our connectivity matrix and then threshold it using an absolute threshold. All of the measures we'll use hear work with either weighted or binary matrices (note that some methods that we don't explore only accept binary matrices).  

In [ ]:
cmat = np.loadtxt('connectivity.csv', delimiter=',')

fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(cmat, vmax=1, vmin=-1, figure=fig)

In [ ]:
thresh_cmat = bct.threshold_absolute(cmat, thr=.4)

fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(thresh_cmat, vmax=1, vmin=-1, figure=fig)

In [ ]:
bin_cmat = bct.binarize(thresh_cmat)
fig, ax = plt.subplots(figsize=(16, 16))
plotting.plot_matrix(bin_cmat, vmax=1, vmin=0, cmap='binary', figure=fig)

## 2. Constructing a graph

Let's just take the same subnetwork we looked at last week, and also add in the motor regions. This will make life a bit easier for understanding various graph theory concepts and measures. Selecting this set of regions from our thresholded matrix:

In [ ]:
A = thresh_cmat[:15, :15]
region_labels = [x[-5:] for x in labels[:15]]

fig, ax = plt.subplots(figsize=(5, 5))
plotting.plot_matrix(A, vmax=1, vmin=-1, labels=region_labels, figure=fig)

In graph theory speak, each region is a **node**. And each corretion between regions/nodes are called **edges**; the strength of the correlation is the edge **weight**. This connectivity structure can be represented in a graph. First we need to build our graph in networkx (`nx`). We can use our connectivity matrix, called an **adjacency matrix** in graph theory (`A`), to create `G`, our graph. Note that we're generating an **undirected weighted** graph because correlations (weights) are nondirectional. 

We'll also label our nodes according to the regions. 

In [ ]:
G = nx.from_numpy_matrix(A)
G = nx.relabel_nodes(G, lambda x: region_labels[x])

### 2.1 Plotting our graph

Now we can plot our graph. A really common way to plot graphs are using force-directed drawing algorithms that try to position nodes such that the edges are roughly equal in length with minimal overlap (see [Wikipedia](https://en.wikipedia.org/wiki/Force-directed_graph_drawing)). Networkx has the Kamada-Kawai algorithm: 

In [ ]:
region_colors = ['purple'] * 9 + ['steelblue'] * 6

nx.draw_kamada_kawai(G, node_color=region_colors, node_size=1000, 
                     with_labels=True, font_color='w')

### 2.2 Adding weights

Above, we're completely ignoring weights/correlations. We can add these in, but we'll have to program this differently so we can customize it a bit.

First, we can extract out our edges and their corresponding weights. This is akin to going through each cell above and getting the corresponding regions, and the correlation value.

In [ ]:
edges, weights = zip(*nx.get_edge_attributes(G,'weight').items())
edges

In [ ]:
weights

In [ ]:
weight_widths = [(.5 + x) ** 4 for x in weights]


layout = nx.kamada_kawai_layout(G)
nx.draw_networkx_nodes(G, layout, node_color=region_colors, node_size=1000, 
                       with_labels=True)
nx.draw_networkx_labels(G, layout, font_color='w')
nx.draw_networkx_edges(G, layout, edgelist=edges, width=weight_widths)
plt.axis('off');

## 3. Graph measures

### 3.1 Network density

We can measure how dense each network is by dividing the number of edges by the number of possible edges. In other words, **network density** is the fraction of possible edges that exist.   

In [ ]:
vis_density, n, k = bct.density_und(A[:9, :9])
mot_density, n, k = bct.density_und(A[9:, 9:])

fig, ax = plt.subplots(figsize=(2, 3))
ax.bar(['Vis', 'Mot'], [vis_density, mot_density],
       width=.8, color=['purple', 'steelblue'])
sns.despine()

By looking at our connectivity matrix, we can confirm these results. For instance, all of the motor regions are connected with one another, hence a density of 1. 

### 3.2 Node degree and strength

**Node degree** is simply the number of edges for a node. If we didn't threshold our connectivity matrix, all nodes would have the same degree (i.e. number of regions - 1). Because we applied a threshold, certain nodes have greater degree than others. We can use Brain Connectivity Toolbox to compute the degrees of our nodes:  

In [ ]:
degrees = bct.degrees_und(A)

fig, ax = plt.subplots(figsize=(5, 3))
ax.bar(region_labels, degrees, width=.8, color=region_colors)
plt.xticks(rotation=90)
sns.despine()

Node degree ignores the connectivity weights (i.e. correlation). **Node strength**, however, is the sum of weights for a node. 

In [ ]:
strengths = bct.strengths_und(A)

fig, ax = plt.subplots(figsize=(5, 3))
ax.bar(region_labels, strengths, width=.8, color=region_colors)
plt.xticks(rotation=90)
sns.despine()

### 3.3 Centrality

Centrality speaks to the extent of which a node is 'central' to the network. There are a variety of measures for centrality, but a particularly popular measure is **eigenvector centrality**.

In [ ]:
eigin_cent = bct.eigenvector_centrality_und(A)

fig, ax = plt.subplots(figsize=(5, 3))
ax.bar(region_labels, eigin_cent, width=.8, color=region_colors)
plt.xticks(rotation=90)
sns.despine()

Alternatively, there is **betweeness centrality**.

In [ ]:
between_cent = bct.betweenness_wei(A)

fig, ax = plt.subplots(figsize=(5, 3))
ax.bar(region_labels, between_cent, width=.8, color=region_colors)
plt.xticks(rotation=90)
sns.despine()

### 3.4 Participation Coefficient

The **participant coefficient** is simply the number of edges that connect to nodes in different clusters/modules (e.g., brain network). So we can measure how much a brain region 'participates' in other brain networks. If the participation coefficient is high, the brain region connects to many other brain networks. Low participation coefficients mean that a brain region mostly (if not entirely) connects to regions within the network it belongs to.   

In [ ]:
network_index = [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2]
participation = bct.participation_coef(A, network_index)

fig, ax = plt.subplots(figsize=(5, 3))
ax.bar(region_labels, participation, width=.8, color=region_colors)
plt.xticks(rotation=90)
sns.despine()

### 3.5 Clustering and Modularity

Modules refer to subgroups (or clusters) of densely connected nodes. We might expect our pre-defined functional brain networks that we already have (Vis, Mot) are their own clusters. We can also apply clustering algorithms or dedicated community-detection algorithms to generate our own communities/clusters/modules. 

We've seen hierarchical clustering before when we did RSA and briefly when we were visualizing connectivity matrices last week. If we apply hierarchical clustering to these data, we get the following:

In [ ]:
# get upper triangle
distances = spatial.distance.squareform(1 - A, checks=False)

# apply hierarchical clustering 
linkages = cluster.hierarchy.linkage(distances, method='average')

# plot dendogram
dendo = cluster.hierarchy.dendrogram(linkages, labels=region_labels)
plt.xticks(rotation=45);

There are other 

In [ ]:
dendo

In [ ]:
modul, q = bct.modularity_und(A)
q

In [ ]:
modul_louvain, q = bct.community_louvain(A)
q

In [ ]:
fig, ax = plt.subplots()
ax.imshow(np.array([modul, modul_louvain]).T, cmap='tab10')
ax.set(yticklabels=region_labels, yticks=range(len(region_labels)), 
       xticklabels=['Modularity', 'Louvain'], xticks=[0, 1]);
plt.xticks(rotation=90)

## 4. Whole-brain analysis

In [ ]:
label_data = pd.DataFrame({'number': np.arange(len(labels)) + 1, 'region': labels})
label_data

In [ ]:
def set_color(x):
    
    networks = ['Vis', 'SomMot', 'DorsAttn', 'SalVentAttn', 
                'Limbic', 'Cont', 'Default']
    cmap = ['purple', 'steelblue', 'green', 'violet', 
            'lightgoldenrodyellow', 'orange', 'indianred']
    pairings = dict(zip(networks, cmap))
    
    network_label = x.split('_')[2]
    return pairings[network_label]

def set_network(x):
    
    networks = ['Vis', 'SomMot', 'DorsAttn', 'SalVentAttn', 
                'Limbic', 'Cont', 'Default']
    index = [1, 2, 3, 4, 5, 6, 7]
    pairings = dict(zip(networks, index))
    
    network_label = x.split('_')[2]
    return pairings[network_label]

node_colors = [set_color(i) for i in labels]
node_network = [set_network(i) for i in labels]

In [ ]:
G = nx.from_numpy_matrix(thresh_cmat)
G = nx.relabel_nodes(G, lambda x: label_data['number'].tolist()[x])



In [ ]:
nx.draw_kamada_kawai(G, node_color=node_colors, node_size=150)

In [ ]:
nx.draw_kamada_kawai(G, node_color=node_colors, node_size=150, with_labels=True)

For reference, the Schaefer atlas layout is below: ![](https://github.com/danjgale/psyc-917/blob/master/images/schaefer_100.png?raw=true)

In [ ]:
def stat_map(x, atlas):

    img_data = atlas.get_fdata()
    indices = np.unique(img_data)[1:]

    arr = img_data.copy()
    for val, i in zip(x, indices):
        arr = np.where(arr == i, val, arr)
        
    return nib.Nifti1Image(arr, atlas.affine)

In [ ]:
A = thresh_cmat

degrees = bct.degrees_und(A)
strengths = bct.strengths_und(A)
eigin_cent = bct.eigenvector_centrality_und(A)
between_cent = bct.betweenness_wei(A)

participation = bct.participation_coef(A, node_network)

In [ ]:
atlas_img = nib.load(atlas['maps'])

res = stat_map(strengths, atlas_img)
plotting.view_img(res, vmin=0, cmap='magma', symmetric_cmap=False)